# Text summarizer AI

## Introduction
[**BART**](https://huggingface.co/docs/transformers/model_doc/bart) (Bidirectional and Auto-Regressive Transformers) is a denoising autoencoder for pretraining sequence-to-sequence models. BART is trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text. 

This notebook uses **bart.large.cnn** model that is pre-trained on English language, and fine-tuned on CNN Daily Mail. Number of parameters is 400M.


### 1. Setup

In [ ]:
# Run this firstly in your kernel if localy, in google colab not necessary


%pip install transformers 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# installing required modules

In [ ]:
# required step


from transformers import pipeline
# importing libs

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", truncation=True)
# Initialize a text summarizer using the Facebook BART model

### 1.1. Scrapers for dataset

This model was trained on [CNN Dailymail Dataset](https://huggingface.co/datasets/cnn_dailymail?row=38).


#### 1.1.1 Scraper for specific URL

In [17]:
import requests
from bs4 import BeautifulSoup

def scrape_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article = ''
    for p in soup.find_all('p'):
        article += p.get_text()
    return article

url = 'https://edition.cnn.com/2023/11/29/world/dutch-translation-british-royal-family-book-withdrawn-scli-intl/index.html'
article = scrape_article(url)
print(article)


      A new book on the British royal family has been pulled from shops in the Netherlands amid reports that a translation error revealed the identity of a family member embroiled in a race row over Prince Harry and Meghan’s first child.
  
      “Endgame,” or “Eindstrijd” in Dutch, the latest book from British writer Omid Scobie was released this week.
  
      The Dutch translation was published in the Netherlands by Xander Uitgevers, which said it was “temporarily withdrawing” the book from sale in a statement on its website Tuesday.
  
      “An error occurred in the Dutch translation and is currently being rectified,” adds the statement.
  
      The Dutch translation reportedly revealed the name of a member of the royal family alleged to have questioned the skin color of Prince Harry and Meghan’s son Archie before he was born, according to PA Media.
  
      In March 2021, the Duchess of Sussex revealed in an interview with Oprah Winfrey that husband Harry told her that there wa

#### 1.1.2. Sraper for all articles in specific year

Source: [fieryflamingo](https://www.fieryflamingo.com/scrape-cnn-homepage-python/)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def url_is_article(url, current_year='2023'):
    if url:
        if 'cnn.com/{}/'.format(current_year) in url and '/gallery/' not in url:
            return True
    return False


def return_text_if_not_none(element):
    if element:
        return element.text.strip()
    else:
        return ''


def parse_timestamp(timestamp):
    if 'Published' in timestamp:
        timestamp_type = 'Published'
    elif 'Updated' in timestamp:
        timestamp_type = 'Updated'
    else:
        timestamp_type = ''

    article_time, article_day, article_year = timestamp.replace('Published', '').replace('Updated', '').split(', ')
    return timestamp_type, article_time.strip(), article_day.strip(), article_year.strip()


def parse(html):
    soup = BeautifulSoup(html, features="html.parser")
    title = return_text_if_not_none(soup.find('h1', {'class': 'headline__text'}))
    author = soup.find('span', {'class': 'byline__name'})
    if not author:
        author = soup.find('span', {'class': 'byline__names'})
    author = return_text_if_not_none(author)
    article_content = return_text_if_not_none(soup.find('div', {'class': 'article__content'}))
    timestamp = return_text_if_not_none(soup.find('div', {'class': 'timestamp'}))
    if timestamp:
        timestamp = parse_timestamp(timestamp)
    else:
        timestamp = ['', '', '', '']
    all_images = soup.find_all('img', {'class': 'image__dam-img'}, src=True)
    all_img_srcs = ''
    if all_images:
        all_img_srcs = [x['src'] for x in all_images if x['src']]
        all_img_srcs = '\n'.join(all_img_srcs)
    data = [title, author, *timestamp, all_img_srcs, article_content]  # the * flattens the tuple
    return [x.strip() for x in data]


all_urls = []
all_data = []
current_year = '2023'
url = 'https://www.cnn.com'
data = requests.get(url).text
soup = BeautifulSoup(data, features="html.parser")
for a in soup.find_all('a', href=True):
    if a['href'] and a['href'][0] == '/' and a['href'] != '#':
        a['href'] = url + a['href']
    all_urls.append(a['href'])


article_urls = [url for url in all_urls if url_is_article(url, current_year=current_year)]
article_urls_duplicates_removed = list(set(article_urls))
print(article_urls_duplicates_removed)
for article_url in article_urls_duplicates_removed:
    article_data = requests.get(article_url).text
    parsed_data = parse(article_data)
    all_data.append(parsed_data)
    print(parsed_data[-1])


df = pd.DataFrame(all_data,
                  columns=['Title', 'Author', 'Timestamp Type', 'Article Year', 'Article Day', 'Article Time', 'Images',
                           'Article Text'])

## 2. Summarization

Summarization of text is one of the abilty of BART model. Bart-large-cnn is trained for summarization

### 2.1. Articles
Here are some dataset examples from various sites (cnn.com, youtube.com, ctk.cz)

In [ ]:
article1 = """NEW DELHI, India (Reuters) -- India has elected its first female president, 
official results show, in what supporters are calling a boost for the rights of millions
of downtrodden women, despite a bitter campaign marked by scandal. Pratibha Patil, 72,
is India's first elected female president. Pratibha Patil, the ruling coalition's 
72-year-old nominee for the mainly ceremonial post, easily beat opposition-backed 
challenger and vice president, Bhairon Singh Shekhawat, in a vote by the national 
parliament and state politicians. "This a victory of the people," Patil told reporters 
after official results were announced Saturday. "I am grateful to the people of India 
and the men and women of India and this is a victory for the principles which our 
Indian people uphold." Patil won about two thirds of the electoral college votes. 
There had never been any doubt she would win, given support from the ruling coalition. 
The governor of the northwestern desert state of Rajasthan, she emerged on the 
national stage when the Congress-led coalition and its communist allies failed to 
agree on a joint candidate. "This is a very special moment for us women, and men of 
course, in our country because for the first time we have a woman being elected 
president of India," Congress party leader Sonia Gandhi, India's most powerful 
politician, said. Supporters hoped Patil's candidacy would help bring issues that 
plague women in India, like dowry-related violence, into the public spotlight. 
A woman is murdered, raped or abused every three minutes on average in India. Her 
presidency also reflects the growing power of some women in India, where an 
increasing number are taking part in the workforce and in schools and hold senior 
positions in corporations. After the results, Patil supporters took to the streets, 
singing and dancing as others lit fire crackers and beat large brass drums. India 
has had a number of female icons in the past -- most famously Sonia Gandhi's 
mother-in-law, Indira, who was one of the world's first female prime ministers in 
1966. But hope Patil's presidency would spark only positive talk about women's 
influence in India evaporated when it emerged the bank for women she helped 
established was closed in 2003 because of bad debts and amid accusations of financial 
irregularities. The employees' union has taken Patil and others to court, claiming 
loans meant for poor women were instead given to her brother and other relatives 
and not returned. She was also accused of trying to shield her brother in a murder 
inquiry. Prime Minister Manmohan Singh, who has dismissed accusations against her 
as "mud-slinging", said on Saturday her victory was "a vote against the politics 
of divisiveness". "All the allegations against me are motivated and have already 
been answered," Patil said in a statement last week. Her campaign was marked by 
other mishaps as well. She managed to offend many minority Muslims, and anger some 
historians, by saying Indian women first veiled their heads as protection against 
16th century Muslim invaders. Then she dismayed modern India by claiming she had 
experienced a "divine premonition" that she was destined for higher office from 
a long dead spiritual guru. Critics also dug up a comment she was said to have made 
as Maharashtra's health minister in 1975, saying people with hereditary diseases 
should be sterilized. E-mail to a friend . Copyright 2007 Reuters. All rights 
reserved.This material may not be published, broadcast, rewritten, or redistributed.
"""
# article from CNN daily mail

article2 = """I'm an IT Admin for infrastructure, this channel was one of the big 
reasons I wanted to do it for a living and this line of Chips, threadripper starting 
at ryzen gen 1 is possibly the single most important piece of hardware for this 
journey. I LOVE the idea of this super overkill option for consumers. If I had the 
money, I would run this in any and all of my computers I and my fam use privately, 
not because they're better for the applications or something but simply because 
they're so cool. I don't quite know what it is but these chips are the only ones 
that excite me quite this much on release. No other piece of hardware has ever 
excited me as much. It's not that I'm an amd fanboy, I recommend Intel chips to 
a lot of people when asked because for most people they're just better, they're 
just the only ones doing hedt and since they've had ryzen and that naming scheme 
for these they've just been dominating
"""
# Youtube comment

article3 = """Bratislava - Český skialpinista dnes nepřežil nehodu ve Vysokých 
Tatrách na Slovensku. Informovala o tom Horská záchranná služba (HZS), podle které 
nezvládl na Štrbském plese jízdu na sjezdovce, když narazil do překážky pod sněhem.
Třiačtyřicetiletý Čech podle HZS po pádu upadl do bezvědomí. Na místo zamířily kromě 
horské služby i týmy leteckých a pozemních záchranářů, ale jejich snaha byla marná.
Horští záchranáři pomáhali dnes odpoledne v Tatrách také při resuscitaci 67letého 
slovenského turisty, který zkolaboval na stezce do Staré Lesné. Ani tohoto muže se 
zachránit nepodařilo.
Lyžařská sezona na Slovensku začíná až o nadcházejícím víkendu.
"""
# Czech article from ctk.cz

article4 = """Praha - Sněmovna dnes hlasy poslanců vládní koalice a přes kritiku 
opozice schválila státní rozpočet na příští rok se schodkem 252 miliard korun. 
Plánovaný schodek má proti tomu letošnímu klesnout o 43 miliard korun. Jde o třetí 
státní rozpočet, který prosadila koaliční vláda premiéra Petra Fialy (ODS). 
Schválený rozpočet nyní dostane k podpisu prezident Petr Pavel. Senát se 
schvalováním rozpočtu nezabývá.
Pro rozpočet hlasovalo 98 poslanců vládní koalice, ke kterým se přidal i nezařazený 
Ivo Vondrák, zvolený původně za opoziční hnutí ANO. Opoziční poslanci hlasovali 
proti. Celkem bylo přihlášeno 181 poslanců.
Poslanci schválili 13 přesunů uvnitř rozpočtu v celkovém objemu 2,7 miliardy korun. 
Schválili například návrh zemědělského výboru na přesun 1,1 miliardy korun na podporu 
vodního hospodářství. Opoziční poslanci se svými návrhy na přesuny peněz neuspěli.
Sněmovna na návrh skupiny koaličních poslanců také přesunula celkem 12,2 milionu 
korun takzvaným paměťovým institucím. Jde například o Masarykovo demokratické hnutí, 
Československou obec legionářskou nebo Spolek pro zachování odkazu českého odboje. 
Jiný schválený návrh skupiny koaličních poslanců přesunuje 120 milionů korun na obnovu 
pevnostních měst Terezín a Josefov.
Z opozičních poslanců neuspěl například Jan Hrnčíř (SPD) s návrhem odebrat 30 miliard 
korun z peněz do rozpočtu EU. Přidat je chtěl na kompenzace pro domácnosti a firmy 
kvůli vyšším cenám energií. Neuspěli ani Karel Havlíček a Alena Schillerová (oba ANO), 
kteří chtěli přidat 27 miliard na obnovitelné zdroje energie nebo přidat 38 miliard 
na zmírnění růstu cen energií.
Rozpočet vychází z předpokladu růstu ekonomiky o 2,3 procenta a růstem spotřeby 
domácností o 3,9 procenta. Na růstu spotřeby domácností by se měl podílet obnovený 
nárůst reálných příjmů. Průměrná míra inflace by měla klesnout z letošních 10,9 na 
2,8 procenta. Plánované příjmy rozpočtu se proti letošku zvýší o 12 miliard na 1,94 
bilionu korun. Plánované výdaje mají naopak klesnout o 31 miliard na 2,19 bilionu 
korun.
Ministr financí Zbyněk Stanjura (ODS) označil rozpočet za odpovědný. Informoval 
poslance mimo jiné o tom, že celkové výdaje na školství vzrostou meziročně o 3,9 
miliardy korun a jejich podíl na celkových výdajích rozpočtu vzrostou na 12 procent. 
Výdaje rozpočtu na zdravotnictví vzrostou o 5,1 miliardy korun a jejich podíl na 
celkových výdajích se meziročně zvýší ze 7,2 na 7,5 procenta. Uvedl také, že na 
investice je určeno 184,2 miliardy korun, což je třetí nejvyšší částka za posledních 
deset let.
Vláda jako hlavní priority rozpočtu uvádí udržení sociálního smíru, zajištění 
předvídatelného způsobu financování zdravotnictví, zajištění platů učitelů na úrovni 
130 procent průměrné mzdy a také zajištění výdajů v objemu dvou procent hrubého 
domácího produktu na obranu. Premiér Petr Fiala (ODS) dnes před poslanci řekl, že 
průměrný plat učitele vzroste v příštím roce na 52.400 korun měsíčně. Průměrná penze 
by od ledna měla vzrůst na 20.635 korun. Výdaje kapitoly Státní dluh se v příštím 
roce zvýší o 25 miliard korun na 95 miliard korun.
"Hospodaření České republiky tak, jak se tady vytvořilo za předcházejících vlád, 
bylo naprosto neudržitelné," prohlásil premiér. Ujišťoval, že vláda šetří na státu, 
a poukazoval na to, že celkové výdaje rozpočtu poprvé od roku 2016 klesnou a klesne 
i počet zaměstnanců státu. Řekl, že vláda je schopna připravit rozpočet, který 
zpomalí růst zadlužení, ale současně je prorůstový a myslí na potřebné investice.
"Je to nejlepší návrh, jakého v současnosti můžeme v podmínkách, které máme, dosáhnout. 
Bojujeme s rekordním zadlužením, s extrémně vysokými nároky - náklady na obsluhu dluhu, 
ale navzdory všem současným výzvám a krizím a zátěži, kterou jsme zdědili po minulé 
vládě, tak jsme opět na cestě k vyrovnanému hospodaření," dodal premiér.
Rozpočet jako obvykle vyvolal kritiku z opozičních lavic. Předseda ANO Andrej Babiš 
naopak Fialu obvinil z toho, že neví, co říká. "My jsme zlepšovali život lidem, my 
jsme předběhli životní úrovní i Itálii, Španělsko. My jsme dneska nejhorší, jsme v 
recesi, největší pokles reálných mezd. Vy tuhle zemi totálně ničíte!" řekl Fialovi. 
Předsedkyně klubu ANO Schillerová řekla, že rozpočet zvyšuje provozní výdaje a současně 
snižuje prorůstové investice. "Není to rozpočet sebevědomé a vzkvétající země s jedním 
z nejnižších veřejných dluhů v Evropské unii, která chce kráčet dopředu a nahoru," 
uvedla. Schillerová nazvala ministra Stanjuru jednotkou arogance, Babiš pro změnu 
jednotkou lhaní.
Předseda SPD Tomio Okamura řekl, že rozpočet ve všech rovinách poškodí Česko, občany 
i firmy. Vládu obvinil mimo jiné z toho, že pokračuje v zadlužování země.
"""
# second czech article from ctk.cz

### 2.2. Summarization examples
Few examples of completed summarization

#### 2.2.1 Summarization of CNN article

In [ ]:
print(summarizer(article1, max_length=200, min_length=30, do_sample=False))

#### 2.2.2 Summarization of YouTube comment

In [ ]:
print(summarizer(article2, max_length=200, min_length=30, do_sample=False))

#### 2.2.3 Summarization of czech article from CTK

In [ ]:
print(summarizer(article3, max_length=200, min_length=30, do_sample=False))

#### 2.2.4 Summarization of second czech article from CTK

In [ ]:
print(summarizer(article4, max_length=200, min_length=30, do_sample=False))